In [ ]:
import panel as pn
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import weather_functions as wf

In [ ]:
w_colors = {'clouds': 'green', 'rain': 'darkblue', 'wind_speed': 'blue', 
            'humidity': 'magenta', 'pressure': 'darkgray', 'temp': 'red'}
fig_height, fig_width = 250, 800
Stations = wf.Stations_SK

In [ ]:
def plot_48h(wdata, val):
    df = wdata['hourly']
    fig = go.Figure()
    plot_function = go.Bar if val == 'rain' else go.Scatter  # this is different from 01_Plotting Weather
    fig.add_trace(plot_function(x=df.index, y=df[val], marker_color=w_colors[val], name=val))
    fig.update_xaxes(dtick=60*60*1000*3, tickformat="%H\n%e.%b")
    fig.update_layout(height=fig_height, width=fig_width)   # plot width and height
    return fig

In [ ]:
temp = {"day": "green", "night": "darkblue", "max": "red", "min": "blue"}

# as in 01_Plotting Weather, but only adds traces to existing Figure fig, which we make with plot_8d
def add_temp_traces(fig, daily, **kwargs):   # **kwargs - variable number of named arguments
    xval = daily.index
    for val in temp:
        fig.add_trace(go.Bar(x=xval, y=daily[val], marker_color=temp[val], name=val), **kwargs)

In [ ]:
def plot_8d(wdata, val):
    df = wdata["daily"]
    fig = go.Figure()
    plot_function = go.Bar if val == 'rain' else go.Scatter
    if val != "temp":
        fig.add_trace(plot_function(x=df.index, y=df[val], marker_color=w_colors[val], name=val))
    else:
        add_temp_traces(fig, df)  # add traces for temperatures throughout the day
    fig.update_xaxes(dtick=60*60*1000*24, tickformat="%e.%b",
                     ticklabelmode="period")
    fig.update_layout(height=fig_height, width=fig_width, showlegend=False)
    return fig

In [ ]:
# general function, mentioned in 01_Plotting_Weather

def plot_forecasts(wdata, period='hourly', values=["temp","rain"]):
    df = wdata[period]
    num_plots = len(values) 
    fig = make_subplots(rows=num_plots, cols=1, subplot_titles=values)

    xval = df.index
    for ind, val in enumerate(values):
        nrow = ind + 1
        if period == 'daily' and val == 'temp':
            add_temp_traces(fig, df, row=nrow, col=1)
        else:
            plot_function = go.Bar if val == 'rain' else go.Scatter
            fig.add_trace(plot_function(x=xval, y=df[val], marker_color=w_colors[val],
                                        name=val), row=nrow, col=1)
        if period == 'hourly':
            fig.update_xaxes(dtick=60*60*1000*3, tickformat="%H\n%e.%b", row=nrow, col=1)
        else:   # daily
            fig.update_xaxes(tickformat="%e.%b", ticklabelmode="period", row=nrow, col=1)
    fig.update_layout(height=num_plots * fig_height, width=fig_width, showlegend=False)
    return fig    